# AutoEnconders

- - -

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
set_matplotlib_formats("retina")

- - -

Comprovamos las versiones de las distintas librerías que importamos

In [ ]:
print("Version de Numpy:", np.__version__)
print("Version de Pandas:", pd.__version__)
import matplotlib as mpl
print("Version de Matplotlib:", mpl.__version__)

- - -

In [ ]:
creditcard = pd.read_csv("https://raw.githubusercontent.com/4data-lab/datasets/master/creditcard.zip")

In [ ]:
print('Número de filas: {}'.format(creditcard.shape[0]))

In [ ]:
print('Número de columnas: {}'.format(creditcard.shape[1]))

In [ ]:
creditcard.head()

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(x="Class", data=creditcard)
plt.xticks((0, 1),('Transacción normal', "Transacción fraudulenta"))

In [ ]:
creditcard["Class"].value_counts()

- - -

## Detección de anomalías

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test = train_test_split(creditcard, test_size=0.20, random_state=1)

In [ ]:
X_train["Class"].value_counts()

In [ ]:
X_test["Class"].value_counts()

- - -

    En "X_train" descartaremos los registros de la clase 1 y también la feature "Class"
    Para el entrenamiento solo usaremos registros no fraudulentos y sin target.    

In [ ]:
X_train = X_train[X_train["Class"] == 0]

In [ ]:
X_train["Class"].value_counts()

In [ ]:
X_train = X_train.drop(["Class"], axis=1)

In [ ]:
type(X_train)

In [ ]:
X_train.to_numpy()

In [ ]:
X_train = X_train.to_numpy()

- - -

Alternativamente podríamos usar el atributo ".values" (ejemplo: X_train.values).

- - -

    Con propósito de prueba en este notebook, de los datos de test crearemos dos subsets cada uno con registros de una clase

In [ ]:
unseen_normal = X_test[X_test["Class"] == 0]

In [ ]:
unseen_normal = unseen_normal.drop(["Class"], axis=1)

In [ ]:
unseen_normal = unseen_normal.to_numpy()

In [ ]:
unseen_fraudulent = X_test[X_test["Class"] == 1]

In [ ]:
unseen_fraudulent = unseen_fraudulent.drop(["Class"], axis=1)

In [ ]:
unseen_fraudulent = unseen_fraudulent.to_numpy()

- - -

    En "y_test" dejaremos registros de ambas clases 0 y 1, pero solo de la feature "Class"

In [ ]:
y_test = X_test["Class"]

In [ ]:
y_test.value_counts()

In [ ]:
type(y_test)

In [ ]:
y_test = y_test.to_numpy()

- - -

    En "X_test" dejaremos registros de ambas clases 0 y 1, pero descartaremos la feature "Class"

In [ ]:
X_test = X_test.drop(["Class"], axis=1)

In [ ]:
X_test = X_test.values

- - -

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
tf.random.set_seed(1)

Miramos si tenemos GPU disponible:

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("No GPU found")

- - -

In [ ]:
X_train.shape[1] #número de features

### Sequential Model

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(14, activation='relu')) 
model.add(tf.keras.layers.Dense(20, activation='relu')) 
model.add(tf.keras.layers.Dense(X_train.shape[1]))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')

Usaremos 'adam' que actualmente es el optimizador de referencia.

Adam (Adaptive Moment Estimation) ajusta la tasa de aprendizaje durante el entrenamiento. De hecho, en scikit learn es el optimizador por defecto.

In [ ]:
model.fit(X_train, X_train, epochs=3)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "my_first_model_with_shape_info.png", show_shapes=True)


- - -

### Functional Model

El modelo secuencial desarrolla el modelo capa por capa como una pila lineal de capas.
Con lo cual es muy fácil construir una red, pero la API secuencial tiene algunas limitaciones que no nos permiten construir modelos que compartan capas o tengan múltiples entradas o salidas.

El modelo funcional es una forma alternativa de construir una red neuronal. 
Proporciona más flexibilidad para desarrollar una red muy compleja con múltiples entradas o salidas, así como un modelo que puede compartir capas.

Veamos cómo implementar con un modelo funcional el mismo modelo que hemos implementado anteriormente con un modelo secuencial:

In [ ]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))
encoder = tf.keras.layers.Dense(20, activation='relu')(inputs)
latent_space = tf.keras.layers.Dense(14, activation='relu')(encoder)
decoder = tf.keras.layers.Dense(20, activation='relu')(latent_space)
outputs = tf.keras.layers.Dense(X_train.shape[1])(decoder)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
with tf.device("/device:GPU:0"):
  model.fit(X_train, X_train, epochs=3)

- - -
    Atención si bien hemos seteado una semilla, los modelos sequencial y funcional no comparten la misma inicialización.
    Aunque son consistentes los resultados de ambos modelos por separado y su código es equivalente, no lo seran los resultados si los comparamos entre sí.
- - -

In [ ]:
X_train_pred = model.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
print("trained_normal (MSE)")
mean_squared_error(X_train_pred,X_train)

In [ ]:
print("unseen_normal (MSE)")
unseen_normal_pred = model.predict(unseen_normal)
mean_squared_error(unseen_normal_pred,unseen_normal)

In [ ]:
print("unseen_fraudulent (MSE)")
unseen_fraudulent_pred = model.predict(unseen_fraudulent)
mean_squared_error(unseen_fraudulent_pred,unseen_fraudulent)

- - -

Alternativamente podemos calcular el error cuadrático medio con Numpy

    sklearn: mean_squared_error(y_true, y_pred)
    numpy:   np.mean(np.square(y_true - y_pred))
             np.mean(np.power(y_true - y_pred))
        

- - -

A continuación obtendremos el error cuadrático medio de cada fila de X_test

In [ ]:
X_test_pred = model.predict(X_test)

In [ ]:
mse = np.mean(np.square(X_test - X_test_pred), axis=1)

In [ ]:
mse.shape

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
precision, recall, umbrales = precision_recall_curve(y_test, mse)
#Recordad: 
#The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
#The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
plt.plot(umbrales, precision[1:], label="Precision")
plt.plot(umbrales, recall[1:], label="Recall")
plt.xlabel('Umbrales')
plt.ylabel('Score')
plt.legend()
plt.grid(True)


In [ ]:
threshold = 10
y_pred = [1 if i > threshold else 0 for i in mse]

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred)

- - -
| *Matriz Confusión*        | Predicción Negativa    | Predicción Positiva     |
| -------------             | :-------------:          | :-------------:           |
| **Observación Negativa** | Verdaderos Negativo (VN)  | Falsos Positivo (FP)      |
| **Observación Positiva** | Falsos Negativo (FN)      | Verdaderos Positivos (VP) |

In [ ]:
#ahora provad a cambiar los thresholds y ver cómo evoluciona la matriz de confusión

- - -